In [32]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [33]:
from Dependencies.random_circuit_generator_universal import * # Using universal set!
from Dependencies.functions_list import *
import psutil, time, numpy as np, csv
from qiskit_aer import StatevectorSimulator
import qiskit.qasm2, qiskit.qasm3
from mqt import ddsim

In [44]:
# @calculate_execution_time
def get_random_circ_h(n: int,h: int, h_prob: float = None):
    if h_prob == None:
        qc, qr, seed = random_circ_h_const(n, h) # has default value set to 0.125
    else:
        qc, qr, seed = random_circ_h_const(n, h, h_prob)
    return qc, qr, seed
def get_random_circ_d(n: int, d: int):
    qc, qr, seed = random_circ_d_const(n, d)
    return qc, qr, seed
def get_random_circ_g(n: int, g: int):
    qc, qr, seed = random_circ_g_const(n, g)
    return qc, qr, seed

""" # Bell State circuit
# qc = QuantumCircuit(2)
# qc.h([0,1])
# qc.cz(0,1)
# qc.h(1)
# d = qc.depth()
"""

' # Bell State circuit\n# qc = QuantumCircuit(2)\n# qc.h([0,1])\n# qc.cz(0,1)\n# qc.h(1)\n# d = qc.depth()\n'

In [45]:
# @calculate_execution_time
def get_stvec_poly(qc, n, t, initial_state):
    if n == t :
        return 
    terms, wire_array, max_new_var = create_poly(qc, n)
    assert t == max_new_var, "Value of 't' != 'max_new_var' from the create_poly function."
    # print("terms are: ", terms)
    # print("wires are: ", wire_array)
    ovs = [j[-1] for j in wire_array]
    # print("Output variables are: ", ovs)
    ttb = get_truthtable(terms, n, t, initial_state)
    # print("ttb is: ", ttb)
    return get_statevector(ttb, n, t, ovs, np)
    # counts = {} # : To-Do

def get_stvec_ddsim(qc):
    backend = ddsim.DDSIMProvider().get_backend("statevector_simulator")
    job = backend.run(qc)
    result = job.result()
    return result.get_statevector()

def get_stvec_aer(qc):
    backend = StatevectorSimulator()
    res = backend.run(qc).result()
    return res.get_statevector()


In [46]:
def get_time_poly(qc, n, t, initial_state):
    # Time Calculation for Simulation using polynomial equation
    start_cpu_times = psutil.Process().cpu_times()
    start_time = time.time()
    # When there is no H gate in our circuit
    if n == t : 
        state_vector = np.zeros(1,dtype=complex)
    else:
        state_vector = get_stvec_poly(qc, n, t, initial_state)

    end_cpu_times = psutil.Process().cpu_times()
    end_time = time.time()

    # Calculate user and system CPU times
    user_time = end_cpu_times.user - start_cpu_times.user
    system_time = end_cpu_times.system - start_cpu_times.system
    cpu_time = user_time + system_time
    wall_time = end_time - start_time

    return state_vector, cpu_time, wall_time

def get_time_ddsim(qc):
    # Time Calculation for Simulation using DDSIM by MQT
    start_cpu_times = psutil.Process().cpu_times()
    start_time = time.time()

    state_vector = get_stvec_ddsim(qc)

    end_cpu_times = psutil.Process().cpu_times()
    end_time = time.time()

    # Calculate user and system CPU times
    user_time = end_cpu_times.user - start_cpu_times.user
    system_time = end_cpu_times.system - start_cpu_times.system
    cpu_time = user_time + system_time
    wall_time = end_time - start_time

    return state_vector, cpu_time, wall_time

def get_time_aer(qc):
    # Time Calculation for Simulation using Qiskit's Aer Simulator
    start_cpu_times = psutil.Process().cpu_times()
    start_time = time.time()

    state_vector = get_stvec_aer(qc)
    # printing the statevector amplitudes with a threshold

    end_cpu_times = psutil.Process().cpu_times()
    end_time = time.time()

    # Calculate user and system CPU times
    user_time = end_cpu_times.user - start_cpu_times.user
    system_time = end_cpu_times.system - start_cpu_times.system
    cpu_time = user_time + system_time
    wall_time = end_time - start_time

    return state_vector, cpu_time, wall_time


In [63]:
def write_results(qc,n,h,h_prob,seed,result):            
    qc_qasm2 = qiskit.qasm2.dumps(qc)
    qc_qasm3 = qiskit.qasm3.dumps(qc)
    qasm2_filename = f'Results/run2/arbitrary_h/qc_qasm2_n{n}_h{h}_h_prob{h_prob}.qasm2'
    qasm3_filename = f'Results/run2/arbitrary_h/qc_qasm3_n{n}_h{h}_h_prob{h_prob}.qasm3'
    with open(qasm2_filename, 'w') as file:
        file.write(f"The seed for the random circuit generator is: {seed}\n")
        file.write(qc_qasm2)
    with open(qasm3_filename, 'w') as file:
        file.write(f"The seed for the random circuit generator is: {seed}\n")
        file.write(qc_qasm3)
    with open('Results/run2/arbitrary_h/program_data_h.csv', 'a', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(result)

In [72]:
# TODO: Something is mismatching. keep it append mode

def check_stvector(n, stvec_poly, stvec_aer, stvec_ddsim, circuit_filename, stvec_comp_filename):
    stvec_ddsim = np.asanyarray(stvec_ddsim)
    stvec_aer = np.asanyarray(stvec_aer)

    with open(stvec_comp_filename, 'a') as result_file:
        for i in range(len(stvec_aer)):
            reversed_index = int(format(i, f'0{n}b')[::-1], 2)
            if stvec_aer[i] != stvec_poly[reversed_index]:
                result_file.write(f"Mismatch found in circuit: {circuit_filename}\n")
                result_file.write(f"Mismatch in stvec_aer at index {i} and stvec_poly (reversed index {reversed_index})\n")
                result_file.write("\n")  
                break

            if stvec_aer[i] != stvec_ddsim[reversed_index]:
                result_file.write(f"Mismatch found in circuit: {circuit_filename}\n")
                result_file.write(f"Mismatch in stvec_aer at index {i} and stvec_ddsim (reversed index {reversed_index})\n")
                result_file.write("\n")  
                break
    print(stvec_poly)
    print(stvec_aer)
    print(stvec_ddsim)

### For varying number of H gates

In [73]:
# If I by-mistake run this cell multiple times, past data will be overwritten, so don't use 'w'.
with open('Results/run2/arbitrary_h/program_data_h.csv', 'a', newline='') as file: 
    writer = csv.writer(file)
    writer.writerow(['n', 'h', 'd', 'g', 't', 'cpu_time_poly', 'wall_time_poly',
                        'cpu_time_ddsim', 'wall_time_ddsim', 'cpu_time_aer', 'wall_time_aer' ])
            
for h_prob in np.arange(5, 6, 2.5):
    for n in range(3,4): 
        for h in range(1,2):
            # print(f"Value of h: {h}")
            h_prob = h_prob/100
            qc, qr, seed = get_random_circ_h(n, h, h_prob)
            # print(qc)
            n = qc.width() 
            h = list(instrct.operation.name for _index, instrct in enumerate(qc.data)).count('h')
            d = qc.depth()
            g = gate_counts(qc)
            t = n + h
            # print(f'Value of n: {n}, d: {d}, g: {g}, h: {h}')
            # Initial state on variables x0 x1 x2 x3 .... continuosly upto n
            # initial_state = [random.randint(0, 1) for _ in range(n)]
            initial_state = [0 for _ in range(n)]
            # print(f'Initial state of qubits is: {initial_state}')

            stvec_poly, cpu_time_poly, wall_time_poly = get_time_poly(qc, n, t, initial_state)
            stvec_aer, cpu_time_aer, wall_time_aer = get_time_aer(qc)
            stvec_ddsim, cpu_time_ddsim, wall_time_ddsim = get_time_ddsim(qc)

            result = [n, h, d, g, t, cpu_time_poly, wall_time_poly,
                        cpu_time_ddsim, wall_time_ddsim, cpu_time_aer, wall_time_aer ]
            # Store the circuit in QASM2 and QASM3 format
            write_results(qc,n,h,h_prob,seed,result)
            circuit_filename = f"qc_qasm3_n{n}_h{h}_h_prob{h_prob}.qasm3"
            stvec_comp_filename = f"Results/run2/arbitrary_h/comparing_statevector_results.txt"
            check_stvector(n, stvec_poly, stvec_aer, stvec_ddsim, circuit_filename, stvec_comp_filename)

[0.70710678+0.j 0.70710678+0.j 0.        +0.j 0.        +0.j
 0.        +0.j 0.        +0.j 0.        +0.j 0.        +0.j]
[0.70710678+0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.70710678+0.j 0.        +0.j 0.        +0.j 0.        +0.j]
[0.70710678+0.j 0.        +0.j 0.        +0.j 0.        +0.j
 0.70710678+0.j 0.        +0.j 0.        +0.j 0.        +0.j]


In [74]:
import signal
import time

# Timeout handler
def timeout_handler(signum, frame):
    raise TimeoutError("Process exceeded time limit")

def your_function():
    # Simulating a long-running task
    time.sleep(10)  # This will take 10 seconds, exceeding the timeout

def run_with_timeout(func, timeout):
    # Set the signal handler to raise TimeoutError after timeout seconds
    signal.signal(signal.SIGALRM, timeout_handler)
    signal.alarm(timeout)  # Set the alarm for the timeout period

    try:
        func()  # Run the function
    except TimeoutError:
        print(f"Process exceeded {timeout} seconds and was terminated.")
    finally:
        signal.alarm(0)  # Disable the alarm

# Run with a timeout of 5 seconds
run_with_timeout(your_function, 5)


Process exceeded 5 seconds and was terminated.


#### Rough Work! Excuse me please.

In [20]:
instructions = [(instruction.operation.name,
                    [qc.find_bit(q).index for q in instruction.qubits]) 
                    for index, instruction in enumerate(qc.data)]
print("instructions of the circuit are: ", instructions)

instructions of the circuit are:  [('h', [0]), ('h', [0]), ('h', [0])]


In [21]:
n = 10
h = 15
qc, qr = get_random_circ(n=n, h=h)
n = qc.width()
h = list(instruction.operation.name for index,
        instruction in enumerate(qc.data)).count('h')
d = qc.depth()
g = gate_counts(qc)
t = n+h
initial_state = [0 for _ in range(n)]
print(gate_counts(qc))
print(qc.count_ops())
# qc.draw(fold=-1)

NameError: name 'get_random_circ' is not defined

In [10]:
stvec_poly, cpu_time_poly, wall_time_poly = get_time_poly(qc, initial_state)
print(cpu_time_poly, wall_time_poly)

Input variables are:  [0, 1, 2, 3, 4, 5, 6, 7, 8, 9]
Output variables are:  [21, 20, 2, 3, 24, 17, 23, 10, 22, 9]
state vector using boolean equation: 
 [0. 0. 0. ... 0. 0. 0.]
3.421875 3.493514060974121


In [38]:
# # Load circuits from QASM file
# filename = "multiple_circuits.qasm"
# circuits = []

# with open(filename, 'r') as file:
#     qasm_code = file.read()

# # Split QASM code based on markers
# circuit_codes = qasm_code.split("//")

# for code in circuit_codes:
#     code = code.strip()
#     if code:
#         circuit = QuantumCircuit.from_qasm_str(code)
#         circuits.append(circuit)

# Now `circuits` list contains all the reconstructed quantum circuits